# Imports

In [414]:
import requests
import pandas as pd
import time
import config
import ast

# <font color = red><b> USER IMPUTS

<b>REQUIRED: </b>`states`, `start_date`, `end_date`<br>

In [415]:
#list of states: use two character abbreviations.
states = ['AL', 'AK', 'AZ', 'AR', 
         # 'CA', 'CO', 'CT', 'DE',
         # 'DC', 'FL', 'GA', 'HI',
         # 'ID', 'IL', 'IN', 'IA',
         # 'KS', 'KY', 'LA', 'ME',
         # 'MD', 'MA', 'MI', 'MN',
         # 'MS', 'MO', 'MT', 'NE', 
         # 'NV', 'NH', 'NJ', 'NM',
         # 'NY', 'NC', 'ND', 'OH',
         # 'OK', 'OR', 'PA', 'RI',
         # 'SC', 'SD', 'TN', 'TX',
         # 'UT', 'VT', 'VA', 'WA',
         # 'WV', 'WI', 'WY'
         ]

#date range: use formatting appropriate for file saving as these values are used in the file names.
start_date = '12-31-2022'
end_date = '12-31-2022'

<b>OPTIONAL:</b> `save_path`

In [454]:
#specify save path for event files csv files
save_path = ''

# 1. Get Events from Ticketmaster Discovery API
---
<font color = red> <b>USER INPUTS:</b></font> `states list`, `start_date`, `end_date`, and `[save_path]`<br>
<b>OUTPUT: for each state: `event-json_[state]_[start_date]_[end_date].csv`</b>  

## –– Function Definitions

Event get request for specific `state`, `date`, `page size`, and `[page number]`

In [457]:
def get_event(state, date, size, KEY, page = None):
    
    #page one of events will not have a specified page number
    if page == None: #use this if request is for the first page
        
        return requests.get(f'https://app.ticketmaster.com/discovery/v2/events.json?stateCode={state}&startDateTime={date}T00:00:00Z&endDateTime={date}T23:59:59Z&size={size}&apikey={KEY}').json()
    
    else: #use this if request is for pages 2+
        
        return requests.get(f'https://app.ticketmaster.com/discovery/v2/events.json?stateCode={state}&startDateTime={date}T00:00:00Z&endDateTime={date}T23:59:59Z&size={size}&page={page}&apikey={KEY}').json()


Generate a list of strings containing a range of dates in 'YYYY-MM-DD' format given `start_date` and `end_date` strings of any format

In [458]:
def generate_date_list(start_date, end_date):
    
    timestamps = pd.date_range(start=start_date, end=end_date)
   
    date_list = []
    
    for i in timestamps:
        
        date_list.append(str(i)[:10])
        
    return date_list

Returns 3 lists containing a list of events, a list of days that had zero events, and a list of days with over 1000 events + the number of events for that day.</br>
Must specify `state`, `start_date`, and `end_date`

In [419]:
def get_events_in_date_list(state, start_date, end_date):
    
    #generate date list given start date and end date
    date_list = generate_date_list(start_date, end_date)
   
    #initiate list objects to return
    event_list = []
    
    dates_with_over_1000_events = []
    
    dates_with_0_events = []
    
    #cycle through each day in the date list
    for date in date_list:
        
        #initiate variables specific to each day
        size = 200
        
        totalElements = 1
        
        totalPages = 1
        
        #try to get first page and throw exception for days with zero events
        try:
            current_get = get_event(state, date, size, config.KEY)
            
            #update total elements and number of pages
            totalElements = current_get['page']['totalElements']
            
            totalPages = current_get['page']['totalPages']
            
            #output for testing: print(f'{date}: totalElements, {totalElements}; totalPages, {totalPages}; size, {size}')
            
            #if date has more than 1000 events, add date to list for alt processing
            if totalElements > 1000:
                
                #output for testing: print(f'{date} has over 1000')
                dates_with_over_1000_events.append([date,totalPages])
                
                break
            
            #output for testing: print(f'page 1')
            
            #add first page to event list
            event_list.extend(current_get['_embedded']['events'])
            
            #cycle through the rest of the pages of events for that day
            for i in range(2,totalPages+1):
                
                #check if loop is on last page
                if i == totalPages:
                    
                    #determine correct size for the last page (if size is not exact api will return no events)
                    size = totalElements % size
                
                #output for testing: print(f'page {i}')
                
                #get page of events
                current_get = get_event(state, date, size, config.KEY, i)
                
                #add events to event lists as individual events
                event_list.extend(current_get['_embedded']['events'])
        
        except:
            #add day to list of days which had no events
            dates_with_0_events.append(date)
    
    return event_list, dates_with_over_1000_events, dates_with_0_events
                      

Given a list of `states`, `start_date`, `end_date`, and `save_path`: <br> • Save a csv containing all events for each state in the list for the given date range. <br>• Print out the dates which had no events, and the days which had more than 1000 events.<i> If one exists, a function will need to be written to split the day up into different time chunks to reduce the number of events in the API call to less than 1000. So far, I have not encountered a need to do this.

In [444]:
def get_events_for_all_states(states, start_date, end_date, save_path = ''):
    
    print(f'Date Range: {start_date} to {end_date}')
    for state in states:
        #get events and create a list of dates that had over 1000 or 0 events
        events, large_dates, zero_dates = get_events_in_date_list(state, start_date, end_date)
        #generate report for each state based on lengths of lists
        total_events = len(events)
        total_large_dates = len(large_dates)
        total_zero_dates = len(zero_dates)
        #report output:
        print('__________________________________________')
        print(state)
        if total_events > 0:
            print(f'    total events: {total_events}')
        if total_large_dates > 0:
            print(f'    1000+ events: {large_dates}')
        if total_zero_dates > 0: 
            print(f'    zero events: {zero_dates}')
        #save json as data frame 
        events_df = pd.DataFrame(events)
        events_df.to_csv(f'{save_path}event-json_{state}_{start_date}_{end_date}.csv')

## –– Function Calls

In [445]:
get_events_for_all_states(states, start_date, end_date, save_path)

Date Range: 12-31-2022 to 12-31-2022
__________________________________________
AL
    total events: 5
__________________________________________
AK
    zero events: ['2022-12-31']
__________________________________________
AZ
    total events: 31
__________________________________________
AR
    total events: 1


# 2. Extract & Flatten Event Information
---
<b>INPUT:</b> for each state: `event-json_[state]_[start_date]_[end_date].csv`<br>
<b>OUTPUT:</b> for each state: `event-expanded_[state]_[start_date]_[end_date].csv`

## –– Function Definitions

### Create Record
For each record, return variables containing flattened features. 
Some information is contained in a string. ast.literal_eval is used to remove outer quotations so the inner dictionaries can be accessed. Some records are missing information. This function checks to see if the information is there, if not, the variable is assigned a None value. For example, if there is no promoter listed, promoter_name will be assigned a None value. 
Some variables are returned which will determine if there are multiple attractions or dmas which will need to be addressed in a following funciton. 
For example, sports games will list two attractions, one for each of the teams playing eachother. 

In [446]:
def create_record(event, attractions_index = 0, classifications_index = 0, venues_index = 0, markets_index = 0, dmas_index = 0):
    # event id & name
    event_id = event['id']
    event_name = event['name']
    #event date
    dates = ast.literal_eval(event['dates'])
    localDate = dates['start']['localDate']
    # does the event span multiple days
    spanMultipleDays = dates['spanMultipleDays']
    #json contains string which needs to be evaluated as code multiple times, this allows the eval to happen only once. 
    embedded = ast.literal_eval(event['_embedded'])
    #event type & subtype
    try: 
        event_classifications: ast.literal_eval(event['classifications'])
        event_type = event_classifications[0]['type']['name']
        event_subType = event_classifications[0]['subType']['name']
    except: 
        event_type = None
        event_subType = None
    #promoter id, name, description
    try:
        promoter = ast.literal_eval(event['promoter'])
        promoter_id = promoter['id']
        promoter_name = promoter['name']
        promoter_description = promoter['description']
    except: 
        promoter_id = None
        promoter_name = None
        promoter_description = None
    #price min & max
    try:
        price = ast.literal_eval(event['priceRanges'])[0]
        price_min = price['min']
        price_max = price['max']
    except: 
        price_min = None
        price_max = None
    #age restrictions 
    try: 
        age_restrictions = ast.literal_eval(event['ageRestrictions'])
        legalAgeEnforced = age_restrictions['legalAgeEnforced']
    except: 
        legalAgeEnforced = None
    #attraction segment, genre, & subgenre
    try: segment = embedded['attractions'][0]['classifications'][0]['segment']['name']
    except: segment = None
    try: genre = embedded['attractions'][0]['classifications'][0]['genre']['name']
    except: genre = None
    try: subGenre = embedded['attractions'][0]['classifications'][0]['subGenre']['name']
    except: subGenre = None
    #venue id & name
    try: venue_id = embedded['venues'][0]['id']
    except: venue_id = None
    try: venue_name = embedded['venues'][0]['name']
    except: venue_name = None
    #venue latitude & longitude
    try: 
        latitude = float(embedded['venues'][0]['location']['latitude'])
        longitude = float(embedded['venues'][0]['location']['longitude'])
    except: 
        latitude = None
        longitude = None
    #venue address
    try: postal_code = embedded['venues'][0]['postalCode']
    except: postal_code = None
    try: state = embedded['venues'][0]['state']['name']
    except: state = None
    try: city = embedded['venues'][0]['city']['name']
    except: city = None
    try: address = embedded['venues'][0]['address']['line1']
    except: address = None
    #venue markets & dmas
    try: market_count = len(embedded['venues'][0]['markets'])
    except: market_count = 0
    try: market = embedded['venues'][0]['markets'][0]['name']
    except: market = None
    try: dma_count = len(embedded['venues'][0]['dmas'])
    except: dma_count = 0
    try: dma = embedded['venues'][0]['dmas'][0]['id']
    except: dma = None
    #attraction info
    try:attraction_count = len(embedded['attractions'])
    except: attraction_count = 0
    try: attraction_id = embedded['attractions'][0]['id']
    except: attraction_id = None
    try: attraction_name = embedded['attractions'][0]['name']
    except: attraction_name = None
    try: attraction_img = embedded['attractions'][0]['images'][0]['url']
    except: attraction_img = None
    #return all relevant variables
    return event_id, event_name, localDate, spanMultipleDays, segment, genre, subGenre, event_type, event_subType, promoter_id, promoter_name, promoter_description, price_min, price_max, legalAgeEnforced, venue_id, venue_name, address, city, state, postal_code, latitude, longitude, market_count, market, dma_count, dma, attraction_count, attraction_id, attraction_name, attraction_img



### Flatten Records
Events only list first attraction. If there is an event with 2 teams playing for example, only one will be listed. A second record will be added in `add_multiple_attractions()` for the second team

In [447]:
def flatten_event_json(state, start_date, end_date, save_path=''):
    #read in event_json csv
    event_json = pd.read_csv(f'{save_path}event-json_{state}_{start_date}_{end_date}.csv')
    #define new dataframe to hold flattened data
    event_flattened = pd.DataFrame(columns = [
                                 'event_id', 
                                 'event_name',
                                 'localDate',
                                 'spanMultipleDays',
                                 'segment',
                                 'genre',
                                 'subGenre',
                                 'event_type',
                                 'event_subType',
                                 'promoter_id',
                                 'promoter_name',
                                 'promoter_description',
                                 'price_min',
                                 'price_max',
                                 'legalAgeEnforced',
                                 'venue_id',
                                 'venue_name',
                                 'address',
                                 'city',
                                 'state',
                                 'postal_code',
                                 'latitude', 
                                 'longitude', 
                                 'market', 
                                 'dma', 
                                 'attraction_id',
                                 'attraction_name', #
                                 'attraction_img' #
                                  ],
                        #determine how many records will be in the dataframe
                        #index = range(0,2))
                        index = range(len(event_json)))
    #initiate dictionaries to track which events have multiples and will need further transformation
    multiple_attractions = {}
    multiple_markets = {}
    multiple_dmas = {}
    #shorter range for troubleshooting:
    #for i in range(2):
    for i in range(len(event_json)):
        #for each event in event_json, extract data and add to flattened dataframe
        try: 
            event_id, event_name, localDate, spanMultipleDays, segment, genre, subGenre, event_type, event_subType, promoter_id, promoter_name, promoter_description, price_min, price_max, legalAgeEnforced, venue_id, venue_name, address, city, state, postal_code, latitude, longitude, market_count, market, dma_count, dma, attraction_count, attraction_id, attraction_name, attraction_img = create_record(event_json.iloc[i])
            event_flattened.iloc[i] = [
                    event_id,
                    event_name,
                    localDate,
                    spanMultipleDays,
                    segment,
                    genre,
                    subGenre,
                    event_type,
                    event_subType,
                    promoter_id,
                    promoter_name,
                    promoter_description,
                    price_min,
                    price_max,
                    legalAgeEnforced,
                    venue_id,
                    venue_name,
                    address,
                    city,
                    state,
                    postal_code,
                    latitude,
                    longitude,
                    market,
                    dma,
                    attraction_id, 
                    attraction_name, 
                    attraction_img
                   ]
            #if there are multiple attractions, markets, or dmas, add the index of that event to the dictionary along with the total number of multiples
            if attraction_count > 1:
                multiple_attractions[i] = attraction_count
            if market_count > 1:
                multiple_markets[i] = market_count
            if dma_count > 1:
                multiple_dmas[i] = dma_count
        except:
            print(f'could not add index:{i}')
    #report output
    print(f'    records: {len(event_flattened)}')
    print(f'    filled records: {i+1}')
    print(f'    multiple attractions: {len(multiple_attractions)}')
    print(f'    attractions to add: {sum(multiple_attractions.values())-len(multiple_attractions)}')
    print(f'    multiple markets: {len(multiple_markets)}')
    print(f'    multiple dmas: {len(multiple_dmas)}')
    return event_json, event_flattened, multiple_attractions, multiple_markets, multiple_dmas

### Add Records With Multiple Attractions
If unique events are desired, group by unique events will be needed in the future. There will be more records than there are events. 

In [448]:
def add_multiple_attractions(multiple_attractions, event_flattened, event_json):
    #initiate event_expanded dataframe that will be saved as initial flattened and expanded csv used in the data cleaning stpes. 
    event_expanded = pd.DataFrame(event_flattened)
    #step through the events listed in the multiple_attractions dictionary and add a row for each
    for i in multiple_attractions:
        for j in range(1,multiple_attractions[i]):
            #attraction id, name & img url
            try: attraction_id = attraction['id']
            except: attraction_id = None
            try: attraction_name = attraction['name']
            except: attraction_name = None
            try: attraction_img = attraction['images'][0]['url']
            except: attraction_img = None
            #attraction segment, genre, subgenre: if the seconds+ attractions do not list a segment, genre or subgenre, use the data from the first attraction
            attraction = ast.literal_eval(event_json.loc[i]['_embedded'])['attractions'][j]
            first_attraction = ast.literal_eval(event_json.loc[i]['_embedded'])['attractions'][0]
            try: segment = attraction['classifications'][0]['segment']['name']
            except:
                try: segment = first_attraction['classifications'][0]['segment']['name']
                except: segment = None
            try: genre = attraction['classifications'][0]['genre']['name']
            except:
                try: genre = first_attracton['classifications'][0]['genre']['name']
                except: genre = None
            try: subGenre = ['classifications'][0]['subGenre']['name']
            except: 
                try: subGenre = first_attraction['classifications'][0]['subGenre']['name']
                except: subGenre = None
            #add new record to the end of the dataframe with new info
            event_expanded.loc[len(event_expanded.index)] = [
                        event_expanded.iloc[i]['event_id'],
                        event_expanded.iloc[i]['event_name'],
                        event_expanded.iloc[i]['localDate'],
                        event_expanded.iloc[i]['spanMultipleDays'],
                        segment,
                        genre,
                        subGenre,
                        event_expanded.iloc[i]['event_type'],
                        event_expanded.iloc[i]['event_subType'],
                        event_expanded.iloc[i]['promoter_id'],
                        event_expanded.iloc[i]['promoter_name'],
                        event_expanded.iloc[i]['promoter_description'],
                        event_expanded.iloc[i]['price_min'],
                        event_expanded.iloc[i]['price_max'],
                        event_expanded.iloc[i]['legalAgeEnforced'],
                        event_expanded.iloc[i]['venue_id'],
                        event_expanded.iloc[i]['venue_name'],
                        event_expanded.iloc[i]['address'],
                        event_expanded.iloc[i]['city'],
                        event_expanded.iloc[i]['state'],
                        event_expanded.iloc[i]['postal_code'],
                        event_expanded.iloc[i]['latitude'],
                        event_expanded.iloc[i]['longitude'],
                        event_expanded.iloc[i]['market'],
                        event_expanded.iloc[i]['dma'],
                        attraction_id, 
                        attraction_name, 
                        attraction_img
                       ]
    #report output:
    print('    ––––––––––––––––––––––––')
    print(f'    attractions added: {len(event_expanded)-len(event_flattened)}')
    print(f'    end records: {len(event_expanded)}')
    return event_expanded

### Expand Events

In [449]:
#list of states that are not able to be expanded
not_expanded = []

def expand_event(state, start_date, end_date, save_path = ''):
    #report output:
    print('––––––––––––––––––––––––––––––––––––')
    print(f'{state}')
    #initiate variables, create expanded version, save to csv.
    try:
        event_json, event_flattened, multiple_attractions, multiple_markets, multiple_dmas = flatten_event_json(state, start_date, end_date, save_path)
        event_expanded = add_multiple_attractions(multiple_attractions, event_flattened, event_json)
        event_expanded.to_csv(f'{save_path}event_expanded_{state}_{start_date}_{end_date}.csv')
    except:
        #add state to list of states that could not be expanded
        not_expanded.append(state)

## –– Function Calls

In [453]:
#report_output
print(f'REPORT for {states} \nDATERANGE: {start_date} to {end_date}')
#step through event_jsons and expand and save each
for state in states:
    expand_event(state, start_date, end_date, save_path)
#report output:   
print(f'––––––––––––––––––––––––––––––––––––\nUnsuccessful: {not_saved}')

REPORT for ['AL', 'AK', 'AZ', 'AR'] 
DATERANGE: 12-31-2022 to 12-31-2022
––––––––––––––––––––––––––––––––––––
AL
    records: 5
    filled records: 5
    multiple attractions: 3
    attractions to add: 3
    multiple markets: 0
    multiple dmas: 5
    ––––––––––––––––––––––––
    attractions added: 3
    end records: 8
––––––––––––––––––––––––––––––––––––
AK
    records: 0
––––––––––––––––––––––––––––––––––––
AZ
    records: 31
    filled records: 31
    multiple attractions: 2
    attractions to add: 2
    multiple markets: 0
    multiple dmas: 25
    ––––––––––––––––––––––––
    attractions added: 2
    end records: 33
––––––––––––––––––––––––––––––––––––
AR
    records: 1
    filled records: 1
    multiple attractions: 0
    attractions to add: 0
    multiple markets: 0
    multiple dmas: 1
    ––––––––––––––––––––––––
    attractions added: 0
    end records: 1
––––––––––––––––––––––––––––––––––––
Unsuccessful: ['AK']


# To Do: Combine expanded events into one csv